In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import glob
import os
%matplotlib inline

In [4]:
print(os.getcwd())
#df_calibration = pd.read_csv()
#plt.plot(df_calibration)

/Users/dcordovacarrizales/Documents/GitHub/Physics_191/Superconductivity


In [ ]:
samples = ['IN','PB2','SNPB','IN2BI','IN4BI', 'SN'] #superconducting
controls = ["PTFE", "Cu"] #non-superconducting

In [ ]:
#### determinr HC at zero temperature and Tc by doing a fit 
files = glob.glob('Meissner/*.txt')
files.sort()

list_samples = []
list_temperatures = []

def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

for file in files:   
    df = pd.read_table(file,names = ['v0','v1','v2','v3'])
    
    #dropping the second half of the sweep down (hysterisis) because the way down often doesn't even have a peak 
    try: 
        row_drop1 = df[df['v0']== max(df['v0'])].index[0]
    except IndexError: 
        row_drop1 = int(len(df['v0'])/2) #if there is an nan error, just takes the half of the length of the total series 

    row_drop2 = len(df['v0'])
    df = df.drop(labels=range(row_drop1, row_drop2), axis=0)
    
    df = df.dropna()
    
    #df['smooth'] = df['v1'].rolling(window=10).mean()
    #df['smooth'] = scipy.signal.savgol_filter(df['v1'], 11, 3 )
    
    #finding second derivative in order to find the peaks in the voltage vs. magnetic field 
    first_derivative = df.diff()
    second_derivative = first_derivative.diff()
    df['2derivative'] = second_derivative['v1']
    #peak_positions = find_peaks(df['2derivative'], height = .001 , threshold = .001, distance = 400 )

    peak_positions = df['2derivative'].idxmin()
    #if len(peak_positions) =! 0:
        #for i in peak_positions: 
            #print(i)
            #plt.plot(df['v0'][i], df['v1'][i], "ro")
    #sample_critical_B = df["v1"].max()
    
    index1_temp = find_nth(file, "_", 2) + 1
    index2_temp = find_nth(file, "K", 1)
    
    index1_name = find_nth(file, "_", 1) + 1 
    index2_name = find_nth(file, "_", 2) 
    
    sample_temperature = file[index1_temp:index2_temp]
    sample_name = file[index1_name: index2_name]

    
    if sample_name == "PTFE" and index2_temp == -1:
        index1_temp = find_nth(file, "_", 2) + 1
        index2_temp = find_nth(file, "T", 2)
        sample_temperature = file[index1_temp:index2_temp]
        
        if float(sample_temperature) == 115e-1: 
            sample_temperature = 1.78
        elif float(sample_temperature) == 450: 
            sample_temperature = 3.7
 
    if index2_temp == -1: 
        index1_temp = find_nth(file, "_", 2) + 1
        index2_temp = find_nth(file, "T", 1)
        sample_temperature = file[index1_temp:index2_temp]

        if float(sample_temperature) == 115e-1: 
            sample_temperature = 1.78
        elif float(sample_temperature) == 450: 
            sample_temperature = 3.7
        
    
    sample_temperature = float(sample_temperature)
    #all_samples[sample_name][0].append(sample_temperature)

    
    list_samples.append(sample_name)
    list_temperatures.append(sample_temperature)
    plt.figure()
    #df.plot(x = 'v0', y = ['v1', 'smooth', '2derivative'],label=['real', 'smooth', '2nd derivative'])
    
    df.plot(x = 'v0', y = ['v1','2derivative'],label=['real','2nd derivative'])

    #for i in range(len(peak_positions[0])):
        #print(i)
        #y = peak_positions[1]['peak_heights'][i]
        #x = peak_positions[0][i]
        #print(y)
    
    plt.plot(df['v0'][peak_positions], df['v1'][peak_positions], marker="o", markersize=5, markerfacecolor= "red", label = df['v0'][peak_positions])
    plt.xlabel("Voltage (V)")
    plt.ylabel("Magnetic Field (T)")
    plt.annotate(str(df['v0'][peak_positions]), (df['v0'][peak_positions], df['v1'][peak_positions]), ha = 'left')
    plt.xlim(left= 0)
    plt.title(sample_name + " at " + str(sample_temperature) + " K")
   

print(list_samples)
print(list_temperatures)

PB0 = [[], []]
PB1 = [[], []]

IN = [[], []]
PB2 = [[], []]
SNPB = [[], []]
Cu = [[], []]
PTFE = [[], []]
IN2BI = [[], []]
IN4BI = [[], []]
SN = [[], []]

all_samples = {"IN": IN, "PB2": PB2, "SNPB": SNPB, "Cu": Cu, "PTFE": PTFE, "IN2BI": IN2BI, "IN4BI": IN4BI, "SN": SN, "PB0": PB0, "PB1":PB1}


#for sample in all_samples: #add some error bars 
    #print(sample)
    #plt.figure()
    #plt.plot(all_samples[sample][0], all_samples[sample][1], "o") 
    #plt.xlabel("Temperature (K)")
    #plt.ylabel("Critical Magnetic Field (T)")
    #plt.title(sample)

In [ ]:
plt.figure()
df = pd.read_table(file,names = ['v0','v1','v2','v3'])
plt.plot(df['v0'],df['v1'],label=file)

In [ ]:
files

In [ ]:
plt.figure()
files = glob.glob('Meissner/6Vpp*PB2*')
for file in files:
    df = pd.read_table(file,names = ['v0','v1','v2','v3'])
    plt.plot(df['v0'],df['v1'])